## Practice 2: Build up a database from scratch and work on it

In [19]:
import sqlite3 as sql
import pathlib
import pandas as pd
import numpy as np

First of all, we do some practice on Pathlib

In [16]:
p = pathlib.Path('/home/drug_discovery/COVID_moonshot_submissions/covid_submissions_all_info.csv')


In [17]:
p

PosixPath('/home/drug_discovery/COVID_moonshot_submissions/covid_submissions_all_info.csv')

In [36]:
pathlib.PurePath('./src/goo/scripts/setup.py').is_absolute()

False

p_1 = pathlib.PurePath('/src/goo')
p_2 = p_1.joinpath('scripts','main.py')
p_2.with_name('not_main.py')

In [48]:
p = pathlib.Path('/home/sabsr3/drug_discovery/COVID_moonshot_submissions')
for child in p.iterdir():
    print(child)
p.exists()
directory = pathlib.Path('/home/sabsr3/drug_discovery/COVID_moonshot_submissions/try')
directory.exists()

/home/sabsr3/drug_discovery/COVID_moonshot_submissions/orders
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/covid_submissions_all_info.csv
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/.git
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/try
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/update_data.py
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/covid_moonshot_ids.csv
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/HTS
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/README.md
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/antiviral_data.xlsx
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/lib
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/.gitignore
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/plots
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/.ipynb_checkpoints
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/Practice_1.py
/home/sabsr3/drug_discovery/COVID_moonshot_submissions/do

True

In [54]:
data_file = Path('COVID_moonshot_submissions/covid_submissions_all_info.csv')
db_file = Path('COVID_moonshot_submissions/sabs_rdbms.db')

In [82]:
# Step 1: drop the database
for table_name in ('compounds','submissions'):
    sql.connect(db_file).execute('DROP TABLE IF EXISTS ' + table_name)

OperationalError: database is locked

In [66]:
# Step 2: create a new database
conn = sql.connect(db_file)
conn.execute('''
CREATE TABLE submissions
(
    submission_id VARCHAR(20) PRIMARY KEY,
    name_code CHAR(3) NOT NULL,
    institute_code CHAR(3) NOT NULL,
    random_id CHAR(8) NOT NULL
)

''')
conn.execute('''
CREATE TABLE compounds
(
    compound_id VARCHAR(20) PRIMARY KEY,
    smiles VARCHAR(2000) NOT NULL,
    submission_id VARCHAR(20) NOT NULL,
    made VARHAR(5) NOT NULL,
    inchi_key CHAR(27) NOT NULL,
    MW DECIMAL NOT NULL,
    r_avg_IC50 DECIMAL,
    FOREIGN KEY(submission_id) REFERENCES submissions(submission_id)
)
''')

In [78]:
# Step 3: Show tables
cur = sql.connect(db_file).execute('''
SELECT name FROM sqlite_master WHERE type = 'table' AND name NOT LIKE 'sqlite_%'
''')
tables = []
for row in cur.fetchall():
    print(row)
    tables.append(row[0])
    
for table in tables:
    cur = sql.connect(db_file).execute('SELECT sql FROM sqlite_master WHERE name=\'' + table + '\'')
    row = cur.fetchone()
    if row is not None:
        print(table+'\n')
        print(row[0])
    else:
        print('Table ' + table + ' not found')

('submissions',)
('compounds',)
submissions

CREATE TABLE submissions
(
    submission_id VARCHAR(20) PRIMARY KEY,
    name_code CHAR(3) NOT NULL,
    institute_code CHAR(3) NOT NULL,
    random_id CHAR(8) NOT NULL
)
compounds

CREATE TABLE compounds
(
    compound_id VARCHAR(20) PRIMARY KEY,
    smiles VARCHAR(2000) NOT NULL,
    submission_id VARCHAR(20) NOT NULL,
    made VARHAR(5) NOT NULL,
    inchi_key CHAR(27) NOT NULL,
    MW DECIMAL NOT NULL,
    r_avg_IC50 DECIMAL,
    FOREIGN KEY(submission_id) REFERENCES submissions(submission_id)
)


In [81]:
# Add records to the table
df = pd.read_csv(data_file)
temp = df['CID'].str.split("-",n=3,expand=True)
submission_table = pd.DataFrame({
    'name_code':temp[0],
    'institute_code':temp[1],
    'random_id':temp[2]
}).drop_duplicates()
submission_table['submission_id'] = submission_table['name_code'] + '-' + submission_table['institute_code'] + '-' + submission_table['random_id']
submission_numpy = submission_table.to_numpy()

conn = sql.connect(db_file)
# Note: the parameters passed into should be Iterable[Iterable[Any]]
conn.executemany("INSERT INTO submissions (name_code,institute_code,random_id,submission_id) VALUES(?,?,?,?)",submission_numpy)

/tmp/ipykernel_4504/4027572663.py:2: DtypeWarning: Columns (3,6,7,9,10,11,12,13,14,15,18,19,27,28,29,30,31,32,33,34,35,36,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,66,67,69,70,71,72,73,74,75,76,84,85,86,87,89,90,91,92,93,94,95,96,104,105,106,107,109,110,111,112,113,114,115,116,124,125,126,127,129,130,131,132,133,134,135,136,144,145,146,147,149,150,151,152,153,154,155,156,164,165,166,167,169,170,171,172,173,174,175,176,184,185,186,187,189,190,191,192,193,194,195,196,204,205,206,207,209,210,211,212,213,214,215,216,224,225,226,227,229,230,231,232,233,234,235,236,244,245,246,247,249,253,254,255,256,257,258,259,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


OperationalError: database is locked